In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import datetime
import logging
import os

In [38]:
# 获取脚本的当前路径，避免计划执行时路径出错
home_dir = os.path.dirname(os.path.realpath(__file__))
# 更换workding directory
working_directory = home_dir
os.chdir(working_directory)

In [39]:
# pip install selenium

In [40]:
# Date
today_date = datetime.date.today() + datetime.timedelta(days=0)
yesterday_date = datetime.date.today() + datetime.timedelta(days=-1)
seven_days_before_date = datetime.date.today() + datetime.timedelta(days=-7)

today_date_string = today_date.strftime('%Y_%m_%d')

In [41]:
# os.getcwd()

In [42]:
log_file_name = 'shopee_add_and_cancel_fans_log\\shopee_add_and_cancel_fans_log_' + today_date_string + '.txt'

# logging.basicConfig(filename=log_file_name, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [43]:
# 屏幕最大化，且指定下载目录
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

prefs = {"profile.default_content_settings.popups": 0,
         "directory_upgrade": True,
         "profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs", prefs)

In [44]:
# 使用chromedriver才可以用开发者权限
chrome_driver_path = ".//chrome_driver//chromedriver.exe"
# browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)

In [45]:
# 所需参数
# 站点；站点后缀；账户；密码；站点top卖家shopid；top卖家username（方便识别）
# shop_list = [['tw', 'tw', 23070969, 'poweradapter.tw', 'kuangyiqiao1991', 9469128, 'alonso.tw'],
#              ['tw', 'tw', 25482220, 'tengus.tw', 'tengus1803', 23068230, 'iu.tw'],
#              ['tw', 'tw', 58707738, 'qianjiaozi', 'qianjiaozi888', 26513957, 'beibei.tw'],
#              ['tw', 'tw', 66377809, 'qianjiaozi1', 'qianjiaozi888', 23068230, 'iu.tw'],
#              ['tw', 'tw', 62416366, 'tengus1.tw', 'tengus1803', 23068230, 'iu.tw'],
#              ['tw', 'tw', 62416544, 'tengus2.tw', 'tengus1803', 23068230, 'iu.tw'],
#              ['tw', 'tw', 63534861, 'qianjiaozitw1', 'tengus1803', 23068230, 'iu.tw'],
#              ['tw', 'tw', 62887142, 'yilanlu.tw', 'yilanlu888', 23481977, 'huoyi.tw'],
#              ['my', 'com.my', 53580963, 'qianjiaozi.my', 'tengus1803', 10891137, 'winners.my'],
#              ['my', 'com.my', 59848325, 'tengus.my', 'tengus1803', 10891137, 'winners.my'],
#              ['my', 'com.my', 62418141, 'tengus1.my', 'tengus1803', 10891137, 'winners.my'],
#              ['my', 'com.my', 62418493, 'tengus2.my', 'tengus1803', 10891137, 'winners.my'],
#              ['id', 'co.id', 59846508, 'tengus.id', 'tengus1803', 28135012, 'sunnyfun.id'],
#              ['id', 'co.id', 62417386, 'tengus1.id', 'tengus1803', 28135012, 'sunnyfun.id'],
#              ['id', 'co.id', 62417551, 'tengus2.id', 'tengus1803', 28135012, 'sunnyfun.id']]

# shop_list = [['tw', 'tw', 23070969, 'poweradapter.tw', 'kuangyiqiao1991', 9469128, 'alonso.tw'],
#              ['th', 'co.th', 117213614, 'tengus.th', 'tengus1803', 25926687, 'xiaozhainv']]

shop_list = [['th', 'co.th', 117213614, 'tengus.th', 'tengus1803', 25926687, 'xiaozhainv'],
             ['sg', 'sg', 182539921, 'tengus1.sg', 'tengus1803', 11918, 'shopeesg'],
             ['ph', 'ph', 182539050, 'tengus.ph', 'tengus1803', 3256461, 'YAZI FASHION ACCESSORIES INC.']]

# shop_list = [['sg', 'sg', 182539921, 'tengus1.sg', 'tengus1803', 11918, 'shopeesg']]
# shop_list = [['ph', 'ph', 182539050, 'tengus.ph', 'tengus1803', 3256461, 'YAZI FASHION ACCESSORIES INC.']]

In [46]:
# 转换为dataframe
shop_df_columns = ['site', 'site_suffix', 'shopid', 'acc', 'pwd', 'top_shop_id', 'top_shop_username']
shop_df = pd.DataFrame(shop_list, columns=shop_df_columns)

In [47]:
# 关注及关注中多语言
following_language = ['关注中', '關注中', 'Following', 'Mengikuti', 'กำลังติดตาม']
not_following_language = ['+ 关注', '+ 關注', '+ Follow', '+ Ikuti', "+ ติดตาม"]

In [48]:
# 取关后关注函数
def add_and_cancel_fans(site, site_suffix, shopid, acc, pwd, top_shop_id, top_shop_username):
    # 打开浏览器
    logging.info('Open the browser.')
    browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)
    
    # main page
    main_page_url = "http://shopee." + site_suffix
    browser.get(main_page_url)
    time.sleep(10)
    
    # remove ads, refresh again
    for i in range(5):
        browser.refresh()
        time.sleep(10)

    # if the shop is on my site, should press the button to choose language
    if site == 'my':
        language_selector = browser.find_elements_by_css_selector('.shopee-button-outline.shopee-button-outline--primary-reverse')
        language_selector[2].click()
        logging.info('Choose Chinese as language.')
        time.sleep(10)
    
    # login
    logging.info('Start to login.')
    LoginElem = browser.find_elements_by_css_selector('.navbar__link.navbar__link--account.navbar__link--tappable.navbar__link--hoverable.navbar__link-text.navbar__link-text--medium')
    
    # 点击登入，弹出账户密码输入框
    LoginElem[1].click()
    time.sleep(10)
    
    # 输入
    acc_password_input_elem = browser.find_elements_by_css_selector('._2QBp41._1b-IZR')
    try:
        acc_password_input_elem[0].click()
        acc_password_input_elem[0].send_keys(acc)
        acc_password_input_elem[1].click()
        acc_password_input_elem[1].send_keys(pwd)
    except:
        time.sleep(30)
        acc_password_input_elem = browser.find_elements_by_css_selector('._2QBp41._1b-IZR')
        acc_password_input_elem[0].click()
        acc_password_input_elem[0].send_keys(acc)
        acc_password_input_elem[1].click()
        acc_password_input_elem[1].send_keys(pwd)
    time.sleep(10)
    
    # 点击提交
    Login_button_elem = browser.find_elements_by_css_selector('._2DvX7K._3j9-lD._3ddytl.SjORHu')
    Login_button_elem[0].click()
    logging.info('Login completed.')
    time.sleep(10)
    
    # switch to my following list
    logging.info('Switch to my following list.')
    my_fans_list_page = 'https://shopee.' + site_suffix + '/shop/' + str(shopid) + '/following/?__classic__=1'
    browser.get(my_fans_list_page)
    
    # cancel certain number of following fans
    to_cancel_num_of_following = 400
    num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
    
    # in case the following number is not greater than 500
    max_roll_time = 100
    roll_time = 0
    
    # page down until we get at least 500 fans to cancel
    while num_of_following_display <= 500 and roll_time <= max_roll_time:
        body = browser.find_element_by_css_selector('body')
        body.send_keys(Keys.END)
        time.sleep(2)
        num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
        logging.info(str(num_of_following_display) + ' following on the list.')
        roll_time += 1
        logging.info('Roll the page for ' + str(roll_time) + ' time.')
    following_buttons = browser.find_elements_by_css_selector('.btn-follow.active.follow.L14')
    
    # actual following to cancel
    to_cancel_num_of_following = min(num_of_following_display, to_cancel_num_of_following)
    
    # page up to the top
    scroll_to_the_top = browser.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.HOME)
    
    # start to cancel following
    logging.info('Start to cancel following.')
    i = 0
    total_cancel_num_of_following = to_cancel_num_of_following
    while i <= total_cancel_num_of_following - 1:
        while to_cancel_num_of_following > 0:
            logging.info('Now run the ' + str(i + 1) + ' time.')
            if following_buttons[i].text in following_language:
                buyer_shopid = following_buttons[i].get_attribute('shopid')
                following_buttons[i].click()
                time.sleep(1)
                to_cancel_num_of_following -= 1
                i += 1
                logging.info(acc + ': ' + str(buyer_shopid) + ' is not following now, ' + str(to_cancel_num_of_following) + ' following remains.')
    
    # add fans from Top Seller's followers
    logging.info('Switch to top seller follower page.')
    top_shop_url = 'https://shopee.' + site_suffix + '/shop/' + str(top_shop_id) + '/followers/?__classic__=1'
    browser.get(top_shop_url)
    
    # add certain number of fans
    to_add_num_of_following = 400
    num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
    
    # page down until we get at least 400 fans to cancel
    logging.info('Start to roll down.')
    while num_of_following_display <= 1000:
        body = browser.find_element_by_css_selector('body')
        # body.send_keys(Keys.PAGE_DOWN)
        body.send_keys(Keys.END)
        time.sleep(2)
        num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
    following_buttons = browser.find_elements_by_css_selector('.btn-follow.follow.L14')
    
    # start to follow buyers from top shop
    logging.info('Start to follow buyers.')
    i = 0
    total_add_num_of_following = to_add_num_of_following
    scroll_to_the_top = browser.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.HOME)

    while i <= total_add_num_of_following - 1:  
        while to_add_num_of_following > 0:
            logging.info('Now run the ' + str(i + 1) + ' time.')
            if following_buttons[i].text in not_following_language:
                # print(following_buttons[i].text)
                buyer_shopid = following_buttons[i].get_attribute('shopid')
                following_buttons[i].click()
                time.sleep(1)
                to_add_num_of_following -= 1
                logging.info(acc + ': ' + str(buyer_shopid) + ' is following now, ' + str(to_add_num_of_following) + ' following remains.')
            else:
                logging.info('Skip this one. It\'s following already.')
            i += 1    
    
    # 关闭
    browser.quit()

In [49]:
# 历遍所有shop
for index, my_shop in shop_df.iterrows():
    site = my_shop[0]
    site_suffix = my_shop[1]
    shopid = my_shop[2]
    acc = my_shop[3]
    pwd = my_shop[4]
    top_shop_id = my_shop[5]
    top_shop_username = my_shop[6]
    
    # 执行
    try:
        logging.info('Now it\'s running the script on ' + acc + '.')
        add_and_cancel_fans(site, site_suffix, shopid, acc, pwd, top_shop_id, top_shop_username)
        logging.info('Finished running the script on ' + acc + '.')
    except Exception as err:
        logging.info('An exception occurred: ' + str(err) + '.')

2019-10-25 22:31:27,513 - INFO - Now it's running the script on tengus.th.
2019-10-25 22:31:27,514 - INFO - Open the browser.
2019-10-25 22:32:44,591 - INFO - Start to login.
2019-10-25 22:33:05,645 - INFO - Login completed.
2019-10-25 22:33:15,646 - INFO - Switch to my following list.
2019-10-25 22:33:28,233 - INFO - 20 following on the list.
2019-10-25 22:33:28,234 - INFO - Roll the page for 1 time.
2019-10-25 22:33:30,275 - INFO - 40 following on the list.
2019-10-25 22:33:30,276 - INFO - Roll the page for 2 time.
2019-10-25 22:33:32,344 - INFO - 40 following on the list.
2019-10-25 22:33:32,345 - INFO - Roll the page for 3 time.
2019-10-25 22:33:34,388 - INFO - 60 following on the list.
2019-10-25 22:33:34,389 - INFO - Roll the page for 4 time.
2019-10-25 22:33:36,459 - INFO - 60 following on the list.
2019-10-25 22:33:36,460 - INFO - Roll the page for 5 time.
2019-10-25 22:33:38,529 - INFO - 80 following on the list.
2019-10-25 22:33:38,532 - INFO - Roll the page for 6 time.
2019-

2019-10-25 22:35:24,776 - INFO - tengus.th: 607536 is not following now, 387 following remains.
2019-10-25 22:35:24,776 - INFO - Now run the 14 time.
2019-10-25 22:35:25,869 - INFO - tengus.th: 167450437 is not following now, 386 following remains.
2019-10-25 22:35:25,869 - INFO - Now run the 15 time.
2019-10-25 22:35:26,962 - INFO - tengus.th: 115311640 is not following now, 385 following remains.
2019-10-25 22:35:26,962 - INFO - Now run the 16 time.
2019-10-25 22:35:28,057 - INFO - tengus.th: 5413793 is not following now, 384 following remains.
2019-10-25 22:35:28,057 - INFO - Now run the 17 time.
2019-10-25 22:35:29,149 - INFO - tengus.th: 8012080 is not following now, 383 following remains.
2019-10-25 22:35:29,149 - INFO - Now run the 18 time.
2019-10-25 22:35:30,243 - INFO - tengus.th: 165448563 is not following now, 382 following remains.
2019-10-25 22:35:30,243 - INFO - Now run the 19 time.
2019-10-25 22:35:31,343 - INFO - tengus.th: 55046857 is not following now, 381 following 

2019-10-25 22:36:24,366 - INFO - tengus.th: 184369211 is not following now, 333 following remains.
2019-10-25 22:36:24,367 - INFO - Now run the 68 time.
2019-10-25 22:36:25,476 - INFO - tengus.th: 7680427 is not following now, 332 following remains.
2019-10-25 22:36:25,476 - INFO - Now run the 69 time.
2019-10-25 22:36:26,578 - INFO - tengus.th: 1838242 is not following now, 331 following remains.
2019-10-25 22:36:26,578 - INFO - Now run the 70 time.
2019-10-25 22:36:27,695 - INFO - tengus.th: 8197760 is not following now, 330 following remains.
2019-10-25 22:36:27,695 - INFO - Now run the 71 time.
2019-10-25 22:36:28,796 - INFO - tengus.th: 36330570 is not following now, 329 following remains.
2019-10-25 22:36:28,796 - INFO - Now run the 72 time.
2019-10-25 22:36:29,915 - INFO - tengus.th: 27395798 is not following now, 328 following remains.
2019-10-25 22:36:29,915 - INFO - Now run the 73 time.
2019-10-25 22:36:31,038 - INFO - tengus.th: 93577541 is not following now, 327 following r

2019-10-25 22:37:23,859 - INFO - tengus.th: 45507617 is not following now, 279 following remains.
2019-10-25 22:37:23,859 - INFO - Now run the 122 time.
2019-10-25 22:37:24,957 - INFO - tengus.th: 114897550 is not following now, 278 following remains.
2019-10-25 22:37:24,957 - INFO - Now run the 123 time.
2019-10-25 22:37:26,049 - INFO - tengus.th: 89043164 is not following now, 277 following remains.
2019-10-25 22:37:26,050 - INFO - Now run the 124 time.
2019-10-25 22:37:27,146 - INFO - tengus.th: 64036580 is not following now, 276 following remains.
2019-10-25 22:37:27,146 - INFO - Now run the 125 time.
2019-10-25 22:37:28,245 - INFO - tengus.th: 43083408 is not following now, 275 following remains.
2019-10-25 22:37:28,245 - INFO - Now run the 126 time.
2019-10-25 22:37:29,334 - INFO - tengus.th: 191524211 is not following now, 274 following remains.
2019-10-25 22:37:29,334 - INFO - Now run the 127 time.
2019-10-25 22:37:30,428 - INFO - tengus.th: 90595240 is not following now, 273 f

2019-10-25 22:38:22,050 - INFO - Now run the 175 time.
2019-10-25 22:38:23,156 - INFO - tengus.th: 186666236 is not following now, 225 following remains.
2019-10-25 22:38:23,157 - INFO - Now run the 176 time.
2019-10-25 22:38:24,250 - INFO - tengus.th: 160984409 is not following now, 224 following remains.
2019-10-25 22:38:24,250 - INFO - Now run the 177 time.
2019-10-25 22:38:25,338 - INFO - tengus.th: 120714810 is not following now, 223 following remains.
2019-10-25 22:38:25,339 - INFO - Now run the 178 time.
2019-10-25 22:38:26,437 - INFO - tengus.th: 157639615 is not following now, 222 following remains.
2019-10-25 22:38:26,437 - INFO - Now run the 179 time.
2019-10-25 22:38:27,537 - INFO - tengus.th: 166281941 is not following now, 221 following remains.
2019-10-25 22:38:27,537 - INFO - Now run the 180 time.
2019-10-25 22:38:28,625 - INFO - tengus.th: 124658276 is not following now, 220 following remains.
2019-10-25 22:38:28,626 - INFO - Now run the 181 time.
2019-10-25 22:38:29,7

...

2019-10-25 22:59:00,248 - INFO - tengus1.sg: 190329041 is not following now, 266 following remains.
2019-10-25 22:59:00,249 - INFO - Now run the 11 time.
2019-10-25 22:59:06,328 - INFO - tengus1.sg: 190328300 is not following now, 265 following remains.
2019-10-25 22:59:06,328 - INFO - Now run the 12 time.
2019-10-25 22:59:12,414 - INFO - tengus1.sg: 190329000 is not following now, 264 following remains.
2019-10-25 22:59:12,414 - INFO - Now run the 13 time.
2019-10-25 22:59:18,507 - INFO - tengus1.sg: 190329603 is not following now, 263 following remains.
2019-10-25 22:59:18,508 - INFO - Now run the 14 time.
2019-10-25 22:59:24,594 - INFO - tengus1.sg: 190332206 is not following now, 262 following remains.
2019-10-25 22:59:24,595 - INFO - Now run the 15 time.
2019-10-25 22:59:30,681 - INFO - tengus1.sg: 190332641 is not following now, 261 following remains.
2019-10-25 22:59:30,682 - INFO - Now run the 16 time.
2019-10-25 22:59:36,770 - INFO - tengus1.sg: 190335080 is not following now,

2019-10-25 23:02:08,703 - INFO - Now run the 64 time.
2019-10-25 23:02:09,787 - INFO - tengus1.sg: 190385034 is not following now, 212 following remains.
2019-10-25 23:02:09,787 - INFO - Now run the 65 time.
2019-10-25 23:02:10,872 - INFO - tengus1.sg: 190385056 is not following now, 211 following remains.
2019-10-25 23:02:10,872 - INFO - Now run the 66 time.
2019-10-25 23:02:11,958 - INFO - tengus1.sg: 190385122 is not following now, 210 following remains.
2019-10-25 23:02:11,958 - INFO - Now run the 67 time.
2019-10-25 23:02:13,038 - INFO - tengus1.sg: 190385165 is not following now, 209 following remains.
2019-10-25 23:02:13,038 - INFO - Now run the 68 time.
2019-10-25 23:02:14,124 - INFO - tengus1.sg: 190385241 is not following now, 208 following remains.
2019-10-25 23:02:14,125 - INFO - Now run the 69 time.
2019-10-25 23:02:15,209 - INFO - tengus1.sg: 190385775 is not following now, 207 following remains.
2019-10-25 23:02:15,209 - INFO - Now run the 70 time.
2019-10-25 23:02:16,29

2019-10-25 23:03:07,357 - INFO - tengus1.sg: 190345722 is not following now, 159 following remains.
2019-10-25 23:03:07,358 - INFO - Now run the 118 time.
2019-10-25 23:03:08,466 - INFO - tengus1.sg: 190345808 is not following now, 158 following remains.
2019-10-25 23:03:08,466 - INFO - Now run the 119 time.
2019-10-25 23:03:09,562 - INFO - tengus1.sg: 190346104 is not following now, 157 following remains.
2019-10-25 23:03:09,562 - INFO - Now run the 120 time.
2019-10-25 23:03:10,657 - INFO - tengus1.sg: 190348479 is not following now, 156 following remains.
2019-10-25 23:03:10,657 - INFO - Now run the 121 time.
2019-10-25 23:03:11,757 - INFO - tengus1.sg: 190346768 is not following now, 155 following remains.
2019-10-25 23:03:11,757 - INFO - Now run the 122 time.
2019-10-25 23:03:12,844 - INFO - tengus1.sg: 190347156 is not following now, 154 following remains.
2019-10-25 23:03:12,844 - INFO - Now run the 123 time.
2019-10-25 23:03:13,929 - INFO - tengus1.sg: 190347340 is not followin

2019-10-25 23:04:05,045 - INFO - tengus1.sg: 190360713 is not following now, 106 following remains.
2019-10-25 23:04:05,045 - INFO - Now run the 171 time.
2019-10-25 23:04:06,132 - INFO - tengus1.sg: 190360914 is not following now, 105 following remains.
2019-10-25 23:04:06,132 - INFO - Now run the 172 time.
2019-10-25 23:04:07,218 - INFO - tengus1.sg: 190361476 is not following now, 104 following remains.
2019-10-25 23:04:07,218 - INFO - Now run the 173 time.
2019-10-25 23:04:08,300 - INFO - tengus1.sg: 190361531 is not following now, 103 following remains.
2019-10-25 23:04:08,301 - INFO - Now run the 174 time.
2019-10-25 23:04:09,384 - INFO - tengus1.sg: 190361710 is not following now, 102 following remains.
2019-10-25 23:04:09,384 - INFO - Now run the 175 time.
2019-10-25 23:04:10,472 - INFO - tengus1.sg: 190361825 is not following now, 101 following remains.
2019-10-25 23:04:10,472 - INFO - Now run the 176 time.
2019-10-25 23:04:11,558 - INFO - tengus1.sg: 190361961 is not followin

2019-10-25 23:05:02,544 - INFO - Now run the 224 time.
2019-10-25 23:05:03,617 - INFO - tengus1.sg: 190370435 is not following now, 52 following remains.
2019-10-25 23:05:03,618 - INFO - Now run the 225 time.
2019-10-25 23:05:04,696 - INFO - tengus1.sg: 190370584 is not following now, 51 following remains.
2019-10-25 23:05:04,696 - INFO - Now run the 226 time.
2019-10-25 23:05:05,767 - INFO - tengus1.sg: 190370601 is not following now, 50 following remains.
2019-10-25 23:05:05,767 - INFO - Now run the 227 time.
2019-10-25 23:05:06,848 - INFO - tengus1.sg: 190370682 is not following now, 49 following remains.
2019-10-25 23:05:06,849 - INFO - Now run the 228 time.
2019-10-25 23:05:07,930 - INFO - tengus1.sg: 190370686 is not following now, 48 following remains.
2019-10-25 23:05:07,930 - INFO - Now run the 229 time.
2019-10-25 23:05:09,005 - INFO - tengus1.sg: 190370854 is not following now, 47 following remains.
2019-10-25 23:05:09,005 - INFO - Now run the 230 time.
2019-10-25 23:05:10,0

In [50]:
# browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)

In [51]:
# browser.get('https://shopee.com.my')

In [52]:
# language_selector = browser.find_elements_by_css_selector('.shopee-button-outline.shopee-button-outline--primary-reverse')
# len(language_selector)

In [53]:
#language_selector[2].click()